# Data loading

##  Loading Data

### Amplifiers

This is an example of how to load data.

In [1]:
from pathlib import Path
from spikeinterface.extractors import IntanRecordingExtractor
from dicarlo_lab_to_nwb.conversion.data_locator import locate_intan_file_path

data_folder = Path("/media/heberto/One Touch/DiCarlo-CN-data-share")
image_set_name = "domain-transfer-2023"
subject = "pico"
session_date = "20230214"
session_time = "140610"

intan_file_path = locate_intan_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)

recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name="RHD2000 amplifier channel",
    all_annotations=True,
    ignore_integrity_checks=False,
)
recording

/Users/yoon/opt/anaconda3/envs/dicarlo_lab_to_nwb_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: module 'scipy.sparse' has no attribute 'csr_matrix'

This particular example has timestamps discontinuities, to load the data regardless we set the parameter `ignore_integrity_checks=True`.

In [ ]:
recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name="RHD2000 amplifier channel",
    all_annotations=True,
    ignore_integrity_checks=True,
)
recording

### Auxiliary input

In [ ]:
recording_auxiliary_input = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name="RHD2000 auxiliary input channel",
    all_annotations=True,
    ignore_integrity_checks=True,
)

recording_auxiliary_input

### ADC input

In [ ]:
recording_adc_input = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name="USB board ADC input channel",
    all_annotations=True,
    ignore_integrity_checks=True,
)

recording_adc_input

### Digital channel 
Requires neo version from github https://github.com/NeuralEnsemble/python-neo/

In [ ]:
recording_digital = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name="USB board digital input channel",
    all_annotations=True,
    ignore_integrity_checks=True,
)

recording_digital

## Loading the probe

In [ ]:
from dicarlo_lab_to_nwb.conversion.probe import build_probe_group
from dicarlo_lab_to_nwb.conversion.data_locator import locate_intan_file_path
from spikeinterface.extractors import IntanRecordingExtractor

data_folder = Path("/media/heberto/One Touch/DiCarlo-CN-data-share")
image_set_name = "domain-transfer-2023"
subject = "pico"
session_date = "20230214"
session_time = "140610"


intan_file_path = locate_intan_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)


stream_name = "RHD2000 amplifier channel"
recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name=stream_name,
    ignore_integrity_checks=True,
    all_annotations=True,
)


probe_group = build_probe_group(recording=recording)


from probeinterface.plotting import plot_probe
import matplotlib.pyplot as plt
import numpy as np



fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(111)

probe = probe_group.probes[0]
channel_ids = recording.get_channel_ids()
corresponding_channel_ids = [channel_ids[i] for i in probe.device_channel_indices]

text_on_contact = np.asarray(corresponding_channel_ids)

plot_probe(probe=probe, ax=ax, with_contact_id=True, text_on_contact=text_on_contact)

In [ ]:
from probeinterface.plotting import plot_probe_group

fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(111)


plot_probe_group(probe_group, ax=ax, same_axes=True, with_contact_id=False)


# Sorting Pipeline

To run a sorting pipeline we need a recording with a geometry attached.

In [ ]:
from spikeinterface.extractors import IntanRecordingExtractor
from spikeinterface.sorters import run_sorter_by_property


from dicarlo_lab_to_nwb.conversion.data_locator import locate_intan_file_path
from dicarlo_lab_to_nwb.conversion.probe import attach_probe_to_recording

data_folder = Path("/media/heberto/One Touch/DiCarlo-CN-data-share")
image_set_name = "domain-transfer-2023"
subject = "pico"
session_date = "20230214"
session_time = "140610"


intan_file_path = locate_intan_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)


stream_name = "RHD2000 amplifier channel"
recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name=stream_name,
    ignore_integrity_checks=True,
    all_annotations=True,
)


attach_probe_to_recording(recording=recording)
recording

Most sorters have been designed with high density probes in mind. They will work with a single channel probe, but the results may not be as good as some units might be supressed by the spatial regularization.

Because of this we performed sorting in two ways so you can compare the results:

1. We do one sorting per probe
2. We do one sorting per channel to avoid interference of the spatial regularization



## Performing a sorting per probe

In [ ]:
from spikeinterface.core import load_extractor

sorting_folder = Path("./sorting_done")
overwrite = False

if sorting_folder.exists() and not overwrite:
    sorting = load_extractor(sorting_folder)
else:
    sorting = run_sorter_by_property(
        sorter_name="kilosort2",
        recording=recording,
        folder="./sorting_folder_probe",
        grouping_property="probe",
        docker_image=True,
    )

    sorting.save(folder=sorting_folder)
    

Action item:
* How to save the sorting results to numpy
* Quality metrics:
    * Which channels are visually driven? we repeat the image 20 times, we randomly choose the first 10 images of the set same of itme
and correlate the responses.
    *   

In [ ]:
sorting

In [ ]:
from spikeinterface.core import create_sorting_analyzer


sorting_analyzer = create_sorting_analyzer(sorting=sorting, recording=recording)




## Performing a sorting per channel

In [ ]:
sorting = run_sorter_by_property(
    sorter_name="kilosort3",
    recording=recording,
    folder="./sorting_folder_per_channel",
    grouping_property="channel_names",
    docker_image=True,
)


In [ ]:
from spikeinterface.sorters import available_sorters

available_sorters()

In [ ]:
from spikeinterface.core import load_extractor

sorting_folder = Path("./sorting_done_per_channel")
overwrite = False

if sorting_folder.exists() and not overwrite:
    sorting = load_extractor(sorting_folder)
else:
    sorting = run_sorter_by_property(
        sorter_name="tridesclous",
        recording=recording,
        folder="./sorting_folder_per_channel",
        grouping_property="channel_names",
        docker_image=True,
    )

    sorting.save(folder=sorting_folder)
    


# Peak Detection Pipeline

## Artificial data

In [ ]:
import spikeinterface.widgets as sw

from spikeinterface.core.generate import generate_ground_truth_recording


recording, sorting = generate_ground_truth_recording(num_channels=4, num_units=1, durations=[1], seed=0)


w_ts = sw.plot_traces(recording, time_range=(0, 1))
w_rs = sw.plot_rasters(sorting, time_range=(0, 1))

In [ ]:
import numpy as np
from dicarlo_lab_to_nwb.conversion.pipeline import di_carlo_peak_detection



job_kwargs = dict(n_jobs=1, verbose=True, progress_bar=True, chunk_duration=1.0)
noise_threshold = 3  # The number of standard deviations for peak detection

spike_times_per_channel = di_carlo_peak_detection(recording=recording, noise_threshold=noise_threshold, job_kwargs=job_kwargs)

In [ ]:
sorting.get_unit_spike_train(0, return_times=True)

In [ ]:
spike_times_per_channel[0]

## Your data

In [ ]:
from pathlib import Path

import spikeinterface.widgets as sw
from dicarlo_lab_to_nwb.conversion.data_locator import locate_intan_file_path

from spikeinterface.extractors import IntanRecordingExtractor


data_folder = Path("/media/heberto/One Touch/DiCarlo-CN-data-share")
image_set_name = "domain-transfer-2023"
subject = "pico"
session_date = "20230214"
session_time = "140610"


intan_file_path = locate_intan_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)



recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name="RHD2000 amplifier channel",
    all_annotations=True,
    ignore_integrity_checks=True,
)

# If you want to select only one channel
channel_ids = recording.get_channel_ids()[0:1]
recording = recording.select_channels(channel_ids=channel_ids)
w_ts = sw.plot_traces(recording, time_range=(0, 1), return_scaled=True)


#### Preprocess

In [ ]:
from dicarlo_lab_to_nwb.conversion.pipeline import DiCarloBandPass, DiCarloNotch


f_notch = 60  # Hz
bandwidth = 10
f_low = 300.0
f_high = 6000.0

vectorized = True 
notched_recording = DiCarloNotch(recording, f_notch=f_notch, bandwidth=bandwidth, vectorized=vectorized)
preprocessed_recording = DiCarloBandPass(notched_recording, f_low=f_low, f_high=f_high, vectorized=vectorized)

# For this instance each array 96 channels, 400 micrometes apart
w_ts = sw.plot_traces(preprocessed_recording, time_range=(0, 1), return_scaled=True)

#### Run the peak detection on a short portion of the data

In [ ]:
from dicarlo_lab_to_nwb.conversion.pipeline import di_carlo_peak_detection

noise_threshold = 3  # The number of standard deviations for peak detection

start_frame = 0
seconds_of_data = 1.0
end_frame = int(preprocessed_recording.sampling_frequency * seconds_of_data)
preprocessed_recording = preprocessed_recording.frame_slice(start_frame=start_frame, end_frame=end_frame)

spike_times_per_channel = di_carlo_peak_detection(
    recording=preprocessed_recording,
    noise_threshold=noise_threshold,
)

spike_times_per_channel

#### Everything can be wrapped up in a couple of lines

In [ ]:
from spikeinterface.extractors import IntanRecordingExtractor
from dicarlo_lab_to_nwb.conversion.pipeline import thresholding_pipeline
from dicarlo_lab_to_nwb.conversion.data_locator import locate_intan_file_path


image_set_name = "domain-transfer-2023"
subject = "pico"
session_date = "20230214"
session_time = "140610"

# Parameters of the pipeline
f_notch = 60  # Hz
bandwidth = 10
f_low = 300.0
f_high = 6000.0
noise_threshold = 3  # The number of standard deviations for peak detection


data_folder = Path("/media/heberto/One Touch/DiCarlo-CN-data-share")

intan_file_path = locate_intan_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)


stream_name = "RHD2000 amplifier channel"
recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name=stream_name,
    ignore_integrity_checks=True,
    all_annotations=True,
)

spike_times_per_channel_vectorized = thresholding_pipeline(
    recording=recording.frame_slice(start_frame=0, end_frame=1000), # Remove frame_slice to run the whole pipeline
    f_notch=f_notch,
    bandwidth=bandwidth,
    f_low=f_low,
    f_high=f_high,
    noise_threshold=noise_threshold,
)

spike_times_per_channel

# Calculating PSTH

In [ ]:
%load_ext autoreload
%autoreload 
from pathlib import Path

from spikeinterface.extractors import IntanRecordingExtractor
from dicarlo_lab_to_nwb.conversion.pipeline import thresholding_pipeline
from dicarlo_lab_to_nwb.conversion.data_locator import locate_intan_file_path
from dicarlo_lab_to_nwb.conversion.probe import attach_probe_to_recording

data_folder = Path("/media/heberto/One Touch/DiCarlo-CN-data-share")
image_set_name = "domain-transfer-2023"
subject = "pico"
session_date = "20230214"
session_time = "140610"

# Parameters of the pipeline
f_notch = 60  # Hz
bandwidth = 10
f_low = 300.0
f_high = 6000.0
noise_threshold = 3  # The number of standard deviations for peak detection

intan_file_path = locate_intan_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)


stream_name = "RHD2000 amplifier channel"
recording = IntanRecordingExtractor(
    file_path=intan_file_path,
    stream_name=stream_name,
    ignore_integrity_checks=True,
    all_annotations=True,
)


attach_probe_to_recording(recording=recording)
chunk_duration = 10.0  # 10 seconds
job_kwargs = dict(n_jobs=-1, progress_bar=True, verbose=True, chunk_duration=chunk_duration)

dict_of_recordings = recording.split_by(property="probe", outputs="dict")
dict_of_spikes_times_per_channel = {}

for probe_name, recording in dict_of_recordings.items():
    spikes_times_per_channel = thresholding_pipeline(
        recording=recording,
        f_notch=f_notch,
        bandwidth=bandwidth,
        f_low=f_low,
        f_high=f_high,
        noise_threshold=noise_threshold,
        vectorized=True,
        job_kwargs=job_kwargs,
    )
    
    dict_of_spikes_times_per_channel[probe_name] = spikes_times_per_channel

# We merge all the dictionaries
dict_of_spikes_times = {key: value for d in dict_of_spikes_times_per_channel.values() for key, value in d.items()}

In [ ]:
import pandas as pd
from dicarlo_lab_to_nwb.conversion.data_locator import locate_mworks_processed_file_path

mworks_processed_file_path = locate_mworks_processed_file_path(
    data_folder=data_folder,
    image_set_name=image_set_name,
    subject=subject,
    session_date=session_date,
    session_time=session_time,
)


mworks_processed_file_path = Path(mworks_processed_file_path)
dtype = {"stimulus_presented": np.uint32, "fixation_correct": bool}
mwkorks_df = pd.read_csv(mworks_processed_file_path, dtype=dtype)
ground_truth_time_column = "samp_on_us"
stimuli_presentation_times_seconds = mwkorks_df[ground_truth_time_column] / 1e6
stimuli_presentation_id = mwkorks_df["stimulus_presented"]
all_stimuli = stimuli_presentation_id.unique()

stimuli_presentation_times_dict = {
    stimulus_id: stimuli_presentation_times_seconds[stimuli_presentation_id == stimulus_id].values for stimulus_id in all_stimuli
}

spike_times_list = [spike_times for spike_times in dict_of_spikes_times.values()]

In [ ]:
from dicarlo_lab_to_nwb.conversion.psth import calculate_event_psth

# Make 10 bins of 0.200 seconds each
number_of_bins = 10
bin_width_in_milliseconds = 400.0 / number_of_bins
#This means the first bin starts 200 ms before the image presentation
milliseconds_from_event_to_first_bin = -200.0  # 
max_repetitions = stimuli_presentation_id.value_counts().max()

# Let's calculate the PSTH for a single stimuli
a_stimuli = all_stimuli[0]
stimulus_presentation_times = stimuli_presentation_times_dict[a_stimuli]

# TODO: Raveling bins in psth.py line 58 assumes increasing order of stimulus presentation times.
# This is not always the case. In case stim A and stim B are presented back-to-back, separated by 
# by an interval near 'milliseconds_from_event_to_first_bin', then the bin values do not always increase.
# @heberto we could add the following, with the cost of losing a small number of repeats,
#  or reconsider the raveling of bins in psth.py
delta_stimulus_times = np.diff(stimulus_presentation_times)
ignore_stim = np.where(delta_stimulus_times < 2*np.abs(milliseconds_from_event_to_first_bin))
if ignore_stim[0].size > 0:
                print(f"Stimulus {stimulus_id} has presentation {ignore_stim[0].size}")
                stimulus_presentation_times = np.delete(stimulus_presentation_times, ignore_stim)

psth_per_stimuli = calculate_event_psth(
    spike_times_list=spike_times_list,
    event_times_seconds=stimulus_presentation_times,
    bin_width_in_milliseconds=bin_width_in_milliseconds,
    number_of_bins = number_of_bins,
    milliseconds_from_event_to_first_bin=milliseconds_from_event_to_first_bin,
    number_of_events=max_repetitions,
)

psth_per_stimuli[1, ...]

### Agregate psth for all stimuli in session

In [ ]:
from tqdm import tqdm
from dicarlo_lab_to_nwb.conversion.psth import calculate_event_psth

number_of_units = len(spike_times_list)
number_of_stimuli = len(stimuli_presentation_times_dict)

session_psth = np.full(
    shape=(number_of_units, number_of_stimuli, max_repetitions, number_of_bins), fill_value=np.nan
)
psth_per_stimuli_dict = {}

for stimulus_index, (stimulus_id, stimulus_times) in enumerate(tqdm(stimuli_presentation_times_dict.items(), desc="Processing Stimuli")):
    psth_per_stimuli = calculate_event_psth(
        spike_times_list=spike_times_list,
        event_times_seconds=stimulus_times,  # make sure this is correct
        bin_width_in_milliseconds=bin_width_in_milliseconds,
        number_of_bins=number_of_bins,
        milliseconds_from_event_to_first_bin=milliseconds_from_event_to_first_bin,
        number_of_events=max_repetitions,
    )
    session_psth[:, stimulus_index, :, :] = psth_per_stimuli
    
session_psth_numpy = session_psth

I also added a numba version that is five times faster

In [ ]:
from tqdm import tqdm
from dicarlo_lab_to_nwb.conversion.psth import calculate_event_psth_numba

number_of_units = len(spike_times_list)
number_of_stimuli = len(stimuli_presentation_times_dict)

session_psth = np.full(
    shape=(number_of_units, number_of_stimuli, max_repetitions, number_of_bins), fill_value=np.nan
)

for stimulus_index, (stimulus_id, stimulus_times) in enumerate(tqdm(stimuli_presentation_times_dict.items(), desc="Processing Stimuli")):
    psth_per_stimuli = calculate_event_psth_numba(
        spike_times_list=spike_times_list,
        event_times_seconds=stimulus_times,  # make sure this is correct
        bin_width_in_milliseconds=bin_width_in_milliseconds,
        number_of_bins=number_of_bins,
        milliseconds_from_event_to_first_bin=milliseconds_from_event_to_first_bin,
        number_of_events=max_repetitions,
    )
    session_psth[:, stimulus_index, :, :] = psth_per_stimuli
    
session_psth_numba = session_psth

In [ ]:
np.allclose(session_psth_numpy, session_psth_numba, equal_nan=True)